<center><h1 style="color:#1a1a1a;
                    font-size:3em">
        Cats vs Dogs With CNN Keras
        </h1> 
        
</center>

<span style="color:grey; font-size:1.2em">Importing Librairies

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

<span style="color:grey; font-size:1.2em">Define Image properties

In [ ]:
Images_Weight = 128
Images_Height = 128
Image_size = (Images_Weight, Images_Height)
Image_Channels=3

<span style="color:grey; font-size:1.2em">Prepare dataset for training model

In [ ]:
filenames = "../input/dogs-vs-cats/train.zip"

categories = []
for f_name in filenames:
  category = f_name.split('.')[0]
  if categories == 'dog':
    categories.append(1)
  else:
    categories.append(0)

df = pd.DataFrame({
    'filename' : filenames,
    'category' : categories
})

<span style="color:grey; font-size:1.2em">Create the neural net model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,\
     Dropout, Flatten,Dense,Activation,\
     BatchNormalization

model = Sequential()
model.add(Conv2D(32,
                 (3,3), activation='relu',input_shape=
                 (Images_Weight, Images_Height, Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

<span style="color:grey; font-size:1.2em">Analyzing the model

In [ ]:
model.summary()

<span style="color:grey; font-size:1.2em">Define callbacks and learning rate

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopping = EarlyStopping(patience = 10)
learning_rate = ReduceLROnPlateau(monitor = "val_acc" , verbose = 1, patience =2, factor = 0.5, min_lr = 0.00001)
callbacks = [earlystopping, learning_rate]

<span style="color:grey; font-size:1.2em">Manage Data

In [ ]:
df["category"] = df["category"].replace({'cat':0, 'dog':1})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df=train_df.reset_index(drop=True)
validate_df=validate_df.reset_index(drop=True)

total_train = train_df.shape[0]
validate_train = validate_df.shape[0]
batch_size = 15